# Hyperparameter Tuning & Model 4 Variations

## Objectives

1. **Optuna Hyperparameter Search**: Systematically optimize Models 1 & 4
2. **Model 4 Variations**: Test alternative knowledge transfer approaches
3. **Final Comparison**: Train winners for 30 epochs and select best model for Kaggle

## The Problem

- **Model 3 (Multimodal)** achieved R² = +0.7537 at 30 epochs BUT cannot be used for Kaggle (requires weather/NDVI/species data)
- **Test set only has images** → We need image-only models
- **Challenge**: How to capture multimodal knowledge in an image-only model?

## Approach

### Models to Optimize

1. **Model 1**: Simple ResNet18 baseline (image only)
2. **Model 4a**: Teacher-Student distillation (learns from multimodal teacher)
3. **Model 4b**: Auxiliary pretraining (learns image→tabular mapping first)

### Timeline

- **Debug mode** (DEBUG_MODE=True): ~30 min (5 trials, 2 epochs)
- **Full run** (DEBUG_MODE=False): ~6-7 hours (50 trials, 30 epochs)

---

## Part 1: Setup & Data Preparation

### Install Required Packages (Run this first!)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm.auto import tqdm
import copy

import optuna
from optuna.trial import TrialState
from optuna.visualization import plot_optimization_history, plot_param_importances

sns.set_style('whitegrid')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✓ Imports complete")

### Import Libraries

In [ ]:
# Install Optuna and visualization dependencies
import sys
!{sys.executable} -m pip install optuna plotly kaleido --quiet
print("✓ Dependencies installed: optuna, plotly, kaleido")

In [ ]:
# ========================
# CONFIGURATION
# ========================

DEBUG_MODE = True  # Set to False for full training

if DEBUG_MODE:
    print("⚠️  DEBUG MODE: Quick testing")
    N_TRIALS = 5           # Optuna trials per model
    EPOCHS_QUICK = 2       # Epochs per trial
    EPOCHS_PHASE1 = 3      # Auxiliary pretraining
    EPOCHS_PHASE2 = 3      # Auxiliary fine-tuning
    EPOCHS_FINAL = 5       # Final winner training
    TIMEOUT = 600          # 10 min timeout
else:
    print("✓ FULL TRAINING MODE")
    N_TRIALS = 50          # Thorough search
    EPOCHS_QUICK = 10      # Good trial evaluation
    EPOCHS_PHASE1 = 15     # Learn image→tabular
    EPOCHS_PHASE2 = 20     # Fine-tune for biomass
    EPOCHS_FINAL = 30      # Full training
    TIMEOUT = 7200         # 2 hour timeout

# Shared config
BATCH_SIZE_DEFAULT = 16
TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
COMPETITION_WEIGHTS = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5])

print(f"\nConfiguration:")
print(f"  N_TRIALS: {N_TRIALS}")
print(f"  EPOCHS_QUICK: {EPOCHS_QUICK}")
print(f"  EPOCHS_FINAL: {EPOCHS_FINAL}")
print(f"  TIMEOUT: {TIMEOUT}s ({TIMEOUT/60:.0f} min)")

In [ ]:
# Load data
train_enriched = pd.read_csv('competition/train_enriched.csv')
train_enriched['Sampling_Date'] = pd.to_datetime(train_enriched['Sampling_Date'])
train_enriched['full_image_path'] = train_enriched['image_path'].apply(lambda x: f'competition/{x}')

# Train/val split
train_data, val_data = train_test_split(train_enriched, test_size=0.2, random_state=42)

print(f"Data loaded: {len(train_data)} train, {len(val_data)} val")
print(f"Targets: {TARGET_COLS}")

In [ ]:
# Calculate target normalization stats (CRITICAL FIX!)
target_means = torch.tensor([train_data[col].mean() for col in TARGET_COLS], dtype=torch.float32)
target_stds = torch.tensor([train_data[col].std() for col in TARGET_COLS], dtype=torch.float32)

print("Target normalization statistics:")
for i, col in enumerate(TARGET_COLS):
    print(f"  {col:15s}: mean={target_means[i]:.2f}g, std={target_stds[i]:.2f}g")

print("\n✓ Will normalize all targets to mean=0, std=1 during training")

In [ ]:
# Prepare tabular features for multimodal/auxiliary training
weather_features = [
    'rainfall_7d', 'rainfall_30d',
    'temp_max_7d', 'temp_min_7d', 'temp_mean_7d', 'temp_mean_30d', 'temp_range_7d',
    'et0_7d', 'et0_30d',
    'water_balance_7d', 'water_balance_30d',
    'days_since_rain', 'daylength', 'season'
]

# Scale continuous features
continuous_features = weather_features + ['Pre_GSHH_NDVI', 'Height_Ave_cm']
scaler = StandardScaler()
train_data[continuous_features] = scaler.fit_transform(train_data[continuous_features])
val_data[continuous_features] = scaler.transform(val_data[continuous_features])

# Encode categorical
le_state = LabelEncoder()
le_species = LabelEncoder()
train_data['State_encoded'] = le_state.fit_transform(train_data['State'])
train_data['Species_encoded'] = le_species.fit_transform(train_data['Species'])
val_data['State_encoded'] = le_state.transform(val_data['State'])
val_data['Species_encoded'] = le_species.transform(val_data['Species'])

num_states = len(le_state.classes_)
num_species = len(le_species.classes_)

print("✓ Tabular features prepared")
print(f"  Weather features: {len(weather_features)}")
print(f"  States: {num_states}")
print(f"  Species: {num_species}")

### Dataset Classes

In [ ]:
class NormalizedDataset(Dataset):
    """Image-only dataset with normalized targets."""
    def __init__(self, dataframe, target_means, target_stds, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds
        
        transform_list = [transforms.Resize((224, 224))]
        
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        self.transform = transforms.Compose(transform_list)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        
        return {
            'image': img,
            'targets': targets_normalized,
            'targets_original': targets
        }

class MultimodalDataset(Dataset):
    """Multimodal dataset for auxiliary pretraining."""
    def __init__(self, dataframe, target_means, target_stds, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds
        
        transform_list = [transforms.Resize((224, 224))]
        
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        self.transform = transforms.Compose(transform_list)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Image
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        
        # Tabular features
        weather = torch.tensor(row[weather_features].values.astype('float32'), dtype=torch.float32)
        ndvi = torch.tensor(row['Pre_GSHH_NDVI'], dtype=torch.float32)
        height = torch.tensor(row['Height_Ave_cm'], dtype=torch.float32)
        state = torch.tensor(row['State_encoded'], dtype=torch.long)
        species = torch.tensor(row['Species_encoded'], dtype=torch.long)
        
        # Targets
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        
        return {
            'image': img,
            'weather': weather,
            'ndvi': ndvi,
            'height': height,
            'state': state,
            'species': species,
            'targets': targets_normalized,
            'targets_original': targets
        }

print("✓ Dataset classes defined")

### Model Architectures

In [ ]:
class SimpleBaseline(nn.Module):
    """Flexible ResNet18 baseline for hyperparameter tuning."""
    def __init__(self, num_outputs=5, hidden_dim=256, dropout=0.2, num_fc_layers=2):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_features = self.resnet.fc.in_features  # 512
        
        # Build flexible FC layers
        layers = []
        in_features = num_features
        
        for i in range(num_fc_layers - 1):
            layers.extend([
                nn.Linear(in_features, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            in_features = hidden_dim
        
        # Final layer
        layers.append(nn.Linear(in_features, num_outputs))
        
        self.resnet.fc = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.resnet(x)
    
    def get_features(self, x):
        """Extract features before final FC (for distillation)."""
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        return x

print("✓ SimpleBaseline defined")

In [ ]:
class MultimodalModel(nn.Module):
    """Multimodal model for teacher in distillation."""
    def __init__(self, num_outputs=5, num_states=4, num_species=15):
        super().__init__()
        
        # Image branch
        self.resnet = models.resnet18(pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        cnn_features = 512
        
        # Weather branch
        self.weather_encoder = nn.Sequential(
            nn.Linear(14, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3)
        )
        
        # Categorical embeddings
        self.state_emb = nn.Embedding(num_states, 8)
        self.species_emb = nn.Embedding(num_species, 16)
        
        # Tabular encoder
        self.tabular_encoder = nn.Sequential(
            nn.Linear(2 + 8 + 16, 32),  # ndvi/height + state + species
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3)
        )
        
        # Fusion head
        total_features = cnn_features + 64 + 32  # 608
        self.fusion_head = nn.Sequential(
            nn.Linear(total_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_outputs)
        )
    
    def forward(self, images, weather, ndvi, height, state, species, return_features=False):
        # Image features
        img_feat = self.resnet(images)
        img_feat = torch.flatten(img_feat, 1)
        
        # Weather features
        weather_feat = self.weather_encoder(weather)
        
        # Tabular features
        state_emb = self.state_emb(state)
        species_emb = self.species_emb(species)
        ndvi_height = torch.stack([ndvi, height], dim=1)
        tabular_input = torch.cat([ndvi_height, state_emb, species_emb], dim=1)
        tabular_feat = self.tabular_encoder(tabular_input)
        
        # Fuse
        combined = torch.cat([img_feat, weather_feat, tabular_feat], dim=1)
        output = self.fusion_head(combined)
        
        if return_features:
            return output, img_feat
        return output

print("✓ MultimodalModel defined")

In [ ]:
class AuxiliaryPretrainedModel(nn.Module):
    """Model 4b: Two-phase training with auxiliary tasks.
    
    Phase 1: Train to predict tabular features from images
    Phase 2: Fine-tune for biomass prediction
    
    At inference: Only needs image (learned implicit tabular patterns)
    """
    def __init__(self, num_outputs=5, hidden_dim=256, dropout=0.2):
        super().__init__()
        
        # Shared backbone: ResNet18
        self.backbone = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-1])
        
        # Phase 1: Auxiliary heads (predict tabular features from image)
        self.ndvi_head = nn.Linear(512, 1)           # Predict NDVI
        self.height_head = nn.Linear(512, 1)         # Predict height
        self.weather_head = nn.Linear(512, 14)       # Predict 14 weather features
        self.state_head = nn.Linear(512, num_states)     # Predict state (classification)
        self.species_head = nn.Linear(512, num_species)  # Predict species (classification)
        
        # Phase 2: Biomass prediction head (used after pretraining)
        self.biomass_head = nn.Sequential(
            nn.Linear(512, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_outputs)
        )
    
    def forward(self, x, mode='biomass'):
        features = self.backbone(x)
        features = features.flatten(1)
        
        if mode == 'auxiliary':
            # Phase 1: Predict tabular features
            return {
                'ndvi': self.ndvi_head(features),
                'height': self.height_head(features),
                'weather': self.weather_head(features),
                'state': self.state_head(features),
                'species': self.species_head(features)
            }
        else:  # mode == 'biomass'
            # Phase 2: Predict biomass
            return self.biomass_head(features)

print("✓ AuxiliaryPretrainedModel defined")

### Utility Functions

In [ ]:
def calculate_competition_r2(predictions, targets):
    """Calculate competition R² (weighted)."""
    per_target_r2 = []
    competition_r2 = 0
    
    weights = COMPETITION_WEIGHTS.numpy()
    
    for i in range(5):
        r2 = r2_score(targets[:, i], predictions[:, i])
        per_target_r2.append(r2)
        competition_r2 += weights[i] * r2
    
    return competition_r2, per_target_r2

class DistillationLoss(nn.Module):
    """Knowledge distillation loss for Model 4a."""
    def __init__(self, temperature=4.0, alpha=0.3, beta=0.5, gamma=0.2):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
    
    def forward(self, student_outputs, student_features, teacher_outputs, teacher_features, targets):
        # Hard loss
        hard_loss = F.mse_loss(student_outputs, targets)
        
        # Soft loss
        soft_loss = F.mse_loss(student_outputs / self.temperature, teacher_outputs / self.temperature)
        
        # Feature loss
        student_feat_norm = F.normalize(student_features, p=2, dim=1)
        teacher_feat_norm = F.normalize(teacher_features, p=2, dim=1)
        feature_loss = 1 - (student_feat_norm * teacher_feat_norm).sum(dim=1).mean()
        
        total_loss = self.alpha * hard_loss + self.beta * soft_loss + self.gamma * feature_loss
        return total_loss

print("✓ Utility functions defined")

---
## Part 2: Optuna Hyperparameter Search

We'll optimize:
- Learning rate
- Weight decay
- Dropout
- Batch size
- Hidden dimensions
- Scheduler type
- Architecture depth

### Model 1: Simple Baseline Optimization

In [ ]:
def train_model_quick(model, train_loader, val_loader, num_epochs, lr, weight_decay, scheduler_type='ReduceLROnPlateau'):
    """Quick training for Optuna trials."""
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Scheduler
    if scheduler_type == 'ReduceLROnPlateau':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    elif scheduler_type == 'CosineAnnealing':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    else:  # StepLR
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=num_epochs//3, gamma=0.1)
    
    best_r2 = -float('inf')
    
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        
        for batch in train_loader:
            images = batch['image'].to(device)
            targets = batch['targets'].to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validate
        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets = batch['targets'].to(device)
                targets_original = batch['targets_original']
                
                outputs = model(images)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * images.size(0)
                
                # Denormalize for R² calculation
                outputs_denorm = outputs.cpu() * target_stds + target_means
                all_preds.append(outputs_denorm.numpy())
                all_targets.append(targets_original.numpy())
        
        val_loss /= len(val_loader.dataset)
        
        # Calculate R²
        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        val_r2, _ = calculate_competition_r2(all_preds, all_targets)
        
        if scheduler_type == 'ReduceLROnPlateau':
            scheduler.step(val_loss)
        else:
            scheduler.step()
        
        best_r2 = max(best_r2, val_r2)
    
    return best_r2

print("✓ train_model_quick() defined")

In [ ]:
def objective_model1(trial):
    """Optuna objective for Model 1."""
    
    # Hyperparameters to tune
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    num_fc_layers = trial.suggest_int('num_fc_layers', 1, 3)
    scheduler_type = trial.suggest_categorical('scheduler', ['ReduceLROnPlateau', 'CosineAnnealing', 'StepLR'])
    
    # Create datasets with trial batch_size
    train_dataset = NormalizedDataset(train_data, target_means, target_stds, augment=True)
    val_dataset = NormalizedDataset(val_data, target_means, target_stds, augment=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # Build model
    model = SimpleBaseline(
        num_outputs=5,
        hidden_dim=hidden_dim,
        dropout=dropout,
        num_fc_layers=num_fc_layers
    )
    
    # Train and return validation R²
    try:
        val_r2 = train_model_quick(model, train_loader, val_loader, EPOCHS_QUICK, lr, weight_decay, scheduler_type)
        return val_r2
    except Exception as e:
        print(f"Trial failed: {e}")
        return -999.0  # Return very bad score on failure

print("✓ objective_model1() defined")

In [ ]:
# Run Optuna study for Model 1
print("\n" + "="*80)
print("OPTUNA HYPERPARAMETER SEARCH: MODEL 1 (SIMPLE BASELINE)")
print("="*80 + "\n")

study1 = optuna.create_study(
    direction='maximize',
    study_name='model1_hyperparam',
    sampler=optuna.samplers.TPESampler(seed=42)
)

study1.optimize(objective_model1, n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True)

print("\n" + "="*80)
print(f"BEST TRIAL: R² = {study1.best_trial.value:+.4f}")
print("="*80)
print("\nBest hyperparameters:")
for key, value in study1.best_params.items():
    print(f"  {key}: {value}")

# Save study
import pickle
with open('optuna_study_model1.pkl', 'wb') as f:
    pickle.dump(study1, f)
print("\n✓ Study saved to: optuna_study_model1.pkl")

In [ ]:
# Visualize Model 1 optimization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Optimization history
ax = axes[0]
trials = study1.trials
trial_numbers = [t.number for t in trials if t.state == TrialState.COMPLETE]
trial_values = [t.value for t in trials if t.state == TrialState.COMPLETE]
ax.plot(trial_numbers, trial_values, 'o-', alpha=0.6)
ax.axhline(y=study1.best_value, color='r', linestyle='--', label=f'Best: {study1.best_value:+.4f}')
ax.set_xlabel('Trial', fontsize=12)
ax.set_ylabel('Validation R²', fontsize=12)
ax.set_title('Model 1: Optimization History', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Parameter importance
ax = axes[1]
importances = optuna.importance.get_param_importances(study1)
params = list(importances.keys())
values = list(importances.values())
ax.barh(params, values)
ax.set_xlabel('Importance', fontsize=12)
ax.set_title('Model 1: Parameter Importance', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('optuna_model1_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved to: optuna_model1_analysis.png")

### Model 4a: Teacher-Student Distillation Optimization

In [ ]:
# First, we need a trained teacher (multimodal model)
# Load the best teacher from previous training

print("Loading pretrained teacher (Model 3 - Multimodal)...")

teacher = MultimodalModel(num_outputs=5, num_states=num_states, num_species=num_species)

# Try to load best teacher weights
import os
if os.path.exists('Model_4_Teacher_best.pth'):
    teacher.load_state_dict(torch.load('Model_4_Teacher_best.pth'))
    print("✓ Loaded existing teacher weights")
elif os.path.exists('Model_3_Multimodal_best.pth'):
    teacher.load_state_dict(torch.load('Model_3_Multimodal_best.pth'))
    print("✓ Loaded existing multimodal weights")
else:
    print("⚠️  No pretrained teacher found, will train one quickly...")
    # Quick teacher training
    teacher = teacher.to(device)
    teacher_dataset_train = MultimodalDataset(train_data, target_means, target_stds, augment=True)
    teacher_dataset_val = MultimodalDataset(val_data, target_means, target_stds, augment=False)
    teacher_loader_train = DataLoader(teacher_dataset_train, batch_size=16, shuffle=True, num_workers=0)
    teacher_loader_val = DataLoader(teacher_dataset_val, batch_size=16, shuffle=False, num_workers=0)
    
    optimizer = torch.optim.AdamW(teacher.parameters(), lr=3e-4, weight_decay=1e-4)
    criterion = nn.MSELoss()
    
    for epoch in range(EPOCHS_QUICK):
        teacher.train()
        for batch in tqdm(teacher_loader_train, desc=f'Training Teacher Epoch {epoch+1}', leave=False):
            images = batch['image'].to(device)
            weather = batch['weather'].to(device)
            ndvi = batch['ndvi'].to(device)
            height = batch['height'].to(device)
            state = batch['state'].to(device)
            species = batch['species'].to(device)
            targets = batch['targets'].to(device)
            
            optimizer.zero_grad()
            outputs = teacher(images, weather, ndvi, height, state, species)
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(teacher.parameters(), max_norm=10.0)
            optimizer.step()
    
    torch.save(teacher.state_dict(), 'teacher_for_optuna.pth')
    print("✓ Teacher trained and saved")

teacher.eval()
teacher = teacher.to(device)
print("✓ Teacher ready for distillation")

In [ ]:
def train_student_quick(student, teacher, train_loader_simple, train_loader_multi, val_loader, 
                       num_epochs, lr, weight_decay, temperature, alpha, beta, gamma):
    """Quick student training with distillation."""
    student = student.to(device)
    teacher.eval()  # Freeze teacher
    
    # Normalize distillation weights
    total = alpha + beta + gamma
    alpha, beta, gamma = alpha/total, beta/total, gamma/total
    
    criterion = DistillationLoss(temperature, alpha, beta, gamma)
    optimizer = torch.optim.AdamW(student.parameters(), lr=lr, weight_decay=weight_decay)
    
    best_r2 = -float('inf')
    
    for epoch in range(num_epochs):
        # Train
        student.train()
        
        for batch_simple, batch_multi in zip(train_loader_simple, train_loader_multi):
            # Student (image only)
            images = batch_simple['image'].to(device)
            targets = batch_simple['targets'].to(device)
            
            student_outputs = student(images)
            student_features = student.get_features(images)
            
            # Teacher (multimodal)
            with torch.no_grad():
                teacher_images = batch_multi['image'].to(device)
                teacher_weather = batch_multi['weather'].to(device)
                teacher_ndvi = batch_multi['ndvi'].to(device)
                teacher_height = batch_multi['height'].to(device)
                teacher_state = batch_multi['state'].to(device)
                teacher_species = batch_multi['species'].to(device)
                
                teacher_outputs, teacher_features = teacher(
                    teacher_images, teacher_weather, teacher_ndvi, teacher_height,
                    teacher_state, teacher_species, return_features=True
                )
            
            # Distillation loss
            loss = criterion(student_outputs, student_features, teacher_outputs, teacher_features, targets)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_norm=10.0)
            optimizer.step()
        
        # Validate
        student.eval()
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets_original = batch['targets_original']
                
                outputs = student(images)
                
                # Denormalize
                outputs_denorm = outputs.cpu() * target_stds + target_means
                all_preds.append(outputs_denorm.numpy())
                all_targets.append(targets_original.numpy())
        
        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        val_r2, _ = calculate_competition_r2(all_preds, all_targets)
        
        best_r2 = max(best_r2, val_r2)
    
    return best_r2

print("✓ train_student_quick() defined")

In [ ]:
def objective_model4(trial):
    """Optuna objective for Model 4a (Teacher-Student)."""
    
    # Student hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    
    # Distillation hyperparameters
    temperature = trial.suggest_float('temperature', 2.0, 8.0)
    alpha = trial.suggest_float('alpha', 0.1, 0.5)  # Hard loss weight
    beta = trial.suggest_float('beta', 0.3, 0.7)    # Soft loss weight
    gamma = trial.suggest_float('gamma', 0.1, 0.3)  # Feature loss weight
    
    # Create datasets
    train_dataset_simple = NormalizedDataset(train_data, target_means, target_stds, augment=True)
    train_dataset_multi = MultimodalDataset(train_data, target_means, target_stds, augment=True)
    val_dataset = NormalizedDataset(val_data, target_means, target_stds, augment=False)
    
    train_loader_simple = DataLoader(train_dataset_simple, batch_size=batch_size, shuffle=True, num_workers=0)
    train_loader_multi = DataLoader(train_dataset_multi, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # Create student
    student = SimpleBaseline(
        num_outputs=5,
        hidden_dim=hidden_dim,
        dropout=dropout,
        num_fc_layers=2
    )
    
    # Train with distillation
    try:
        val_r2 = train_student_quick(
            student, teacher, train_loader_simple, train_loader_multi, val_loader,
            EPOCHS_QUICK, lr, weight_decay, temperature, alpha, beta, gamma
        )
        return val_r2
    except Exception as e:
        print(f"Trial failed: {e}")
        return -999.0

print("✓ objective_model4() defined")

In [ ]:
# Run Optuna study for Model 4a
print("\n" + "="*80)
print("OPTUNA HYPERPARAMETER SEARCH: MODEL 4A (TEACHER-STUDENT)")
print("="*80 + "\n")

study4 = optuna.create_study(
    direction='maximize',
    study_name='model4_distillation',
    sampler=optuna.samplers.TPESampler(seed=42)
)

study4.optimize(objective_model4, n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True)

print("\n" + "="*80)
print(f"BEST TRIAL: R² = {study4.best_trial.value:+.4f}")
print("="*80)
print("\nBest hyperparameters:")
for key, value in study4.best_params.items():
    print(f"  {key}: {value}")

# Save study
with open('optuna_study_model4.pkl', 'wb') as f:
    pickle.dump(study4, f)
print("\n✓ Study saved to: optuna_study_model4.pkl")

In [ ]:
# Visualize Model 4a optimization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Optimization history
ax = axes[0]
trials = study4.trials
trial_numbers = [t.number for t in trials if t.state == TrialState.COMPLETE]
trial_values = [t.value for t in trials if t.state == TrialState.COMPLETE]
ax.plot(trial_numbers, trial_values, 'o-', alpha=0.6)
ax.axhline(y=study4.best_value, color='r', linestyle='--', label=f'Best: {study4.best_value:+.4f}')
ax.set_xlabel('Trial', fontsize=12)
ax.set_ylabel('Validation R²', fontsize=12)
ax.set_title('Model 4a: Optimization History', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Parameter importance
ax = axes[1]
importances = optuna.importance.get_param_importances(study4)
params = list(importances.keys())
values = list(importances.values())
ax.barh(params, values)
ax.set_xlabel('Importance', fontsize=12)
ax.set_title('Model 4a: Parameter Importance', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('optuna_model4_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved to: optuna_model4_analysis.png")

### Optuna Summary

In [ ]:
print("\n" + "="*80)
print("OPTUNA HYPERPARAMETER SEARCH SUMMARY")
print("="*80 + "\n")

summary_df = pd.DataFrame({
    'Model': ['1. Simple Baseline', '4a. Teacher-Student'],
    'Best R²': [f"{study1.best_value:+.4f}", f"{study4.best_value:+.4f}"],
    'Trials Completed': [len([t for t in study1.trials if t.state == TrialState.COMPLETE]),
                         len([t for t in study4.trials if t.state == TrialState.COMPLETE])],
    'Best LR': [f"{study1.best_params['lr']:.2e}", f"{study4.best_params['lr']:.2e}"],
    'Best Dropout': [f"{study1.best_params['dropout']:.2f}", f"{study4.best_params['dropout']:.2f}"]
})

print(summary_df.to_string(index=False))
print("\n" + "="*80)

In [ ]:
# Update todo list
from IPython.display import Markdown
Markdown("""
## ✓ Section 2 Complete: Optuna Hyperparameter Search

**Completed:**
- Model 1 optimization ({} trials)
- Model 4a optimization ({} trials)
- Visualization and analysis

**Next:** Model 4b - Auxiliary Pretraining
""".format(len(study1.trials), len(study4.trials)))

---
## Part 3: Model 4b - Auxiliary Pretraining

**Hypothesis**: If we train the model to predict tabular features (NDVI, height, weather, species) from images first, it will learn visual patterns that correlate with those features. Then when we fine-tune for biomass, the model can leverage this implicit understanding.

**Two-Phase Training:**
1. **Phase 1**: Train image → tabular (NDVI, height, weather, state, species)
2. **Phase 2**: Fine-tune for biomass prediction

**At inference**: Only needs image (learned to "see" tabular patterns)

### Phase 1: Auxiliary Pretraining (Image → Tabular)

In [ ]:
def auxiliary_loss(predictions, targets):
    """Multi-task loss for predicting tabular features."""
    loss_ndvi = F.mse_loss(predictions['ndvi'].squeeze(), targets['ndvi'])
    loss_height = F.mse_loss(predictions['height'].squeeze(), targets['height'])
    loss_weather = F.mse_loss(predictions['weather'], targets['weather'])
    loss_state = F.cross_entropy(predictions['state'], targets['state'])
    loss_species = F.cross_entropy(predictions['species'], targets['species'])
    
    # Weighted combination
    total_loss = (
        0.2 * loss_ndvi + 
        0.2 * loss_height + 
        0.3 * loss_weather + 
        0.15 * loss_state + 
        0.15 * loss_species
    )
    
    return total_loss, {
        'ndvi': loss_ndvi.item(),
        'height': loss_height.item(),
        'weather': loss_weather.item(),
        'state': loss_state.item(),
        'species': loss_species.item()
    }

print("✓ auxiliary_loss() defined")

In [ ]:
def train_auxiliary_phase1(model, train_loader, val_loader, num_epochs):
    """Phase 1: Train to predict tabular features from images."""
    print("\n" + "="*80)
    print("MODEL 4B - PHASE 1: AUXILIARY PRETRAINING (Image → Tabular)")
    print("="*80 + "\n")
    
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
    
    history = {'train_loss': [], 'val_loss': [], 'val_ndvi_mae': [], 'val_state_acc': []}
    
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        
        for batch in tqdm(train_loader, desc=f'Phase 1 Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = {
                'ndvi': batch['ndvi'].to(device),
                'height': batch['height'].to(device),
                'weather': batch['weather'].to(device),
                'state': batch['state'].to(device),
                'species': batch['species'].to(device)
            }
            
            optimizer.zero_grad()
            predictions = model(images, mode='auxiliary')
            loss, _ = auxiliary_loss(predictions, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validate
        model.eval()
        val_loss = 0
        ndvi_errors = []
        state_correct = 0
        state_total = 0
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets = {
                    'ndvi': batch['ndvi'].to(device),
                    'height': batch['height'].to(device),
                    'weather': batch['weather'].to(device),
                    'state': batch['state'].to(device),
                    'species': batch['species'].to(device)
                }
                
                predictions = model(images, mode='auxiliary')
                loss, _ = auxiliary_loss(predictions, targets)
                val_loss += loss.item() * images.size(0)
                
                # NDVI MAE
                ndvi_errors.extend((predictions['ndvi'].squeeze().cpu() - targets['ndvi'].cpu()).abs().tolist())
                
                # State accuracy
                state_pred = predictions['state'].argmax(dim=1)
                state_correct += (state_pred == targets['state']).sum().item()
                state_total += targets['state'].size(0)
        
        val_loss /= len(val_loader.dataset)
        ndvi_mae = np.mean(ndvi_errors)
        state_acc = state_correct / state_total
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_ndvi_mae'].append(ndvi_mae)
        history['val_state_acc'].append(state_acc)
        
        print(f"Epoch {epoch+1:2d}/{num_epochs}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, "
              f"NDVI MAE={ndvi_mae:.4f}, State Acc={state_acc:.2%}")
    
    print(f"\n✓ Phase 1 complete! Model learned to predict tabular features from images.\n")
    return history

print("✓ train_auxiliary_phase1() defined")

In [ ]:
# Train Model 4b Phase 1
model_4b = AuxiliaryPretrainedModel(num_outputs=5, hidden_dim=256, dropout=0.2)

# Create multimodal dataloaders
train_dataset_multi = MultimodalDataset(train_data, target_means, target_stds, augment=True)
val_dataset_multi = MultimodalDataset(val_data, target_means, target_stds, augment=False)
train_loader_multi = DataLoader(train_dataset_multi, batch_size=BATCH_SIZE_DEFAULT, shuffle=True, num_workers=0)
val_loader_multi = DataLoader(val_dataset_multi, batch_size=BATCH_SIZE_DEFAULT, shuffle=False, num_workers=0)

history_4b_phase1 = train_auxiliary_phase1(model_4b, train_loader_multi, val_loader_multi, EPOCHS_PHASE1)

# Save Phase 1 weights
torch.save(model_4b.state_dict(), 'model_4b_phase1.pth')
print("✓ Phase 1 weights saved to: model_4b_phase1.pth")

### Phase 2: Fine-tune for Biomass Prediction

In [ ]:
def train_auxiliary_phase2(model, train_loader, val_loader, num_epochs, freeze_backbone=False):
    """Phase 2: Fine-tune for biomass prediction."""
    print("\n" + "="*80)
    print("MODEL 4B - PHASE 2: BIOMASS FINE-TUNING")
    print("="*80 + "\n")
    
    model = model.to(device)
    
    if freeze_backbone:
        print("⚠️  Freezing backbone (only train biomass head)")
        for param in model.backbone.parameters():
            param.requires_grad = False
        optimizer = torch.optim.AdamW(model.biomass_head.parameters(), lr=3e-4, weight_decay=1e-4)
    else:
        print("✓ Fine-tuning entire model (backbone with low LR)")
        optimizer = torch.optim.AdamW([
            {'params': model.backbone.parameters(), 'lr': 1e-5},
            {'params': model.biomass_head.parameters(), 'lr': 3e-4}
        ], weight_decay=1e-4)
    
    criterion = nn.MSELoss()
    history = {'train_loss': [], 'val_loss': [], 'val_r2': [], 'epoch': []}
    best_r2 = -float('inf')
    
    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        
        for batch in tqdm(train_loader, desc=f'Phase 2 Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = batch['targets'].to(device)
            
            optimizer.zero_grad()
            outputs = model(images, mode='biomass')
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validate
        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets = batch['targets'].to(device)
                targets_original = batch['targets_original']
                
                outputs = model(images, mode='biomass')
                loss = criterion(outputs, targets)
                val_loss += loss.item() * images.size(0)
                
                # Denormalize
                outputs_denorm = outputs.cpu() * target_stds + target_means
                all_preds.append(outputs_denorm.numpy())
                all_targets.append(targets_original.numpy())
        
        val_loss /= len(val_loader.dataset)
        
        # Calculate R²
        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        val_r2, _ = calculate_competition_r2(all_preds, all_targets)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_r2'].append(val_r2)
        history['epoch'].append(epoch + 1)
        
        print(f"Epoch {epoch+1:2d}/{num_epochs}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val R²={val_r2:+.4f}")
        
        if val_r2 > best_r2:
            best_r2 = val_r2
            torch.save(model.state_dict(), 'model_4b_phase2_best.pth')
            print(f"  💾 New best R²={best_r2:+.4f}")
    
    print(f"\n✓ Phase 2 complete! Best R²={best_r2:+.4f}\n")
    return history, best_r2

print("✓ train_auxiliary_phase2() defined")

In [ ]:
# Train Model 4b Phase 2
# Create image-only dataloaders
train_dataset_simple = NormalizedDataset(train_data, target_means, target_stds, augment=True)
val_dataset_simple = NormalizedDataset(val_data, target_means, target_stds, augment=False)
train_loader_simple = DataLoader(train_dataset_simple, batch_size=BATCH_SIZE_DEFAULT, shuffle=True, num_workers=0)
val_loader_simple = DataLoader(val_dataset_simple, batch_size=BATCH_SIZE_DEFAULT, shuffle=False, num_workers=0)

history_4b_phase2, best_r2_4b = train_auxiliary_phase2(
    model_4b, train_loader_simple, val_loader_simple, EPOCHS_PHASE2, freeze_backbone=False
)

print(f"\n✓ Model 4b training complete!")
print(f"  Phase 1: Learned image→tabular mapping")
print(f"  Phase 2: Fine-tuned for biomass, R²={best_r2_4b:+.4f}")

### Compare Model 4 Variants

In [ ]:
print("\n" + "="*80)
print("MODEL 4 VARIANTS COMPARISON")
print("="*80 + "\n")

model4_comparison = pd.DataFrame({
    'Model': ['4a. Teacher-Student', '4b. Auxiliary Pretrain'],
    'Best R² (Optuna)': [f"{study4.best_value:+.4f}", f"{best_r2_4b:+.4f}"],
    'Approach': ['Knowledge distillation', 'Multi-task pretraining'],
    'Training': ['Student learns from teacher', 'Phase 1: Image→tabular, Phase 2: Biomass'],
    'Advantage': ['Soft targets from teacher', 'Explicit tabular-visual alignment']
})

print(model4_comparison.to_string(index=False))
print("\n" + "="*80)

if best_r2_4b > study4.best_value:
    print(f"\n🏆 Model 4b (Auxiliary) wins! R²={best_r2_4b:+.4f} vs {study4.best_value:+.4f}")
    print("   → Auxiliary pretraining successfully transfers tabular knowledge!")
    best_model4_variant = '4b'
    best_model4_r2 = best_r2_4b
else:
    print(f"\n🏆 Model 4a (Distillation) wins! R²={study4.best_value:+.4f} vs {best_r2_4b:+.4f}")
    print("   → Traditional distillation is more effective for this task")
    best_model4_variant = '4a'
    best_model4_r2 = study4.best_value

In [ ]:
# Visualize Model 4b training
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Phase 1: Auxiliary training
ax = axes[0]
epochs_p1 = list(range(1, len(history_4b_phase1['train_loss']) + 1))
ax.plot(epochs_p1, history_4b_phase1['train_loss'], 'o-', label='Train Loss', linewidth=2)
ax.plot(epochs_p1, history_4b_phase1['val_loss'], 's-', label='Val Loss', linewidth=2)
ax2 = ax.twinx()
ax2.plot(epochs_p1, history_4b_phase1['val_ndvi_mae'], '^-', color='green', label='NDVI MAE', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)
ax2.set_ylabel('NDVI MAE', fontsize=12, color='green')
ax.set_title('Model 4b Phase 1: Image→Tabular Training', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
ax.grid(alpha=0.3)

# Phase 2: Biomass training
ax = axes[1]
ax.plot(history_4b_phase2['epoch'], history_4b_phase2['val_r2'], 'o-', linewidth=2)
ax.axhline(y=best_r2_4b, color='r', linestyle='--', label=f'Best: {best_r2_4b:+.4f}')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Validation R²', fontsize=12)
ax.set_title('Model 4b Phase 2: Biomass Fine-tuning', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('model_4b_training.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved to: model_4b_training.png")

---
## Part 4: Final Training & Comparison (30 Epochs)

Now we'll train the optimized models for the full duration and compare results.

In [ ]:
# This section will be populated based on your decision to proceed with 30-epoch training
# For now, let's create a summary of what we have

print("\n" + "="*80)
print("FINAL SUMMARY: READY FOR 30-EPOCH TRAINING")
print("="*80 + "\n")

final_summary = pd.DataFrame({
    'Model': [
        '1. Simple (Optimized)',
        f'4{best_model4_variant}. Best Model 4',
        '3. Multimodal (Reference)'
    ],
    f'R² ({EPOCHS_QUICK} epochs)': [
        f"{study1.best_value:+.4f}",
        f"{best_model4_r2:+.4f}",
        'N/A'
    ],
    'R² (30 epochs)': [
        'TODO: Train with best params',
        'TODO: Train with best params',
        '+0.7537 (from notebook 11)'
    ],
    'Can Submit?': ['✅ Yes', '✅ Yes', '❌ No (needs tabular)'],
    'Hyperparameters': ['Optuna optimized', 'Optuna/default', 'Default']
})

print(final_summary.to_string(index=False))
print("\n" + "="*80)

print("\n📋 Next Steps:")
print("  1. Set DEBUG_MODE=False at the top of notebook")
print("  2. Re-run Optuna searches (50 trials each, ~4 hours)")
print("  3. Train Model 1 with best params for 30 epochs (~1 hour)")
print(f"  4. Train Model {best_model4_variant} with best params for 30 epochs (~1 hour)")
print("  5. Compare final results and select winner for Kaggle")
print("\n✓ Notebook structure complete!")

---
## Notebook Complete!

### What We've Built:

1. **Optuna Hyperparameter Search** for Models 1 & 4a
2. **Model 4b**: Auxiliary pretraining approach (image→tabular→biomass)
3. **Comparison framework** to evaluate all approaches

### Key Findings (Debug Mode):

- Best Model 1 hyperparameters found
- Best Model 4a distillation parameters found  
- Model 4b auxiliary pretraining tested
- All models are image-only and can be used for Kaggle submission

### To Run Full Experiment:

```python
# Set at top of notebook:
DEBUG_MODE = False
```

Then:
- Run full Optuna search (50 trials, 10 epochs each)
- Train winners for 30 epochs
- Select best model for Kaggle test predictions

### Expected Final Results:

- **Model 1 (optimized)**: R² ~ 0.70-0.75
- **Model 4 (best variant)**: R² ~ 0.65-0.72
- **Winner**: Likely Model 1 (simplicity + optimization)

---